Autores Gabriel Conejo Valerio -2014093542 Nasser Brown Joseph Jimenez Zuñiga - 2016133677

In [ ]:
from sklearn.preprocessing import OneHotEncoder

MLP (Perceptron Multicapa)

In [ ]:
class PerceptronMulticapa:
    def __init__(self, capas, alpha=0.1):
        self.capas = capas
        self.alpha = alpha
        self.bias = []
        self.pesos = []
        for i in range(0, len(capas) - 1):
            # Inicializar los pesos y bias de cada capa
            peso = np.random.randn(capas[i], capas[i+1])
            self.pesos.append(peso)
            bias = np.random.randn(capas[i+1])
            self.bias.append(bias)
    def activacion(self, x):
        # Funcion de activacion sigmoide
        return 1.0 / (1 + np.exp(-x))
    def activacion_derivada(self, x):
        # Derivada de la funcion de activacion sigmoide
        return x * (1 - x)
    def feedforward(self, X):
        # Calcular la salida de cada capa
        capa_activacion = [X]
        for i in range(0, len(self.capas) - 1):
            x = np.dot(capa_activacion[i], self.pesos[i]) + self.bias[i]
            y = self.activacion(x)
            capa_activacion.append(y)
        return capa_activacion
    def backpropagation(self, X, y, capa_activacion):
        # Calcular el error de la capa de salida
        error = capa_activacion[-1] - y
        deltas = [error]
        # Propagar el error hacia atras a traves de la red neuronal
        for i in reversed(range(1, len(self.capas) - 1)):
            delta = deltas[-1]
            activacion = capa_activacion[i]
            activacion_anterior = capa_activacion[i-1]
            delta_anterior = np.dot(delta, self.pesos[i].T)
            deltas.append(delta_anterior * self.activacion_derivada(activacion))
        deltas = list(reversed(deltas))
        for i in range(len(self.capas) - 1):
            activacion_anterior = capa_activacion[i]
            delta = deltas[i]
            d_peso = np.outer(activacion_anterior, delta)
            d_bias = delta
            self.pesos[i] -= self.alpha * d_peso
            self.bias[i] -= self.alpha * d_bias
    def entrenar(self, X, y, epochs):
        for epoch in range(0, epochs):
            for i in range(0, len(X)):
                capa_activacion = self.feedforward(X[i])
                self.backpropagation(X[i], y[i], capa_activacion)
    def predecir(self, X):
        # Obtener la salida de la ultima capa
        capa_activacion = self.feedforward(X)
        return capa_activacion[-1]
    
def encode_labels(y):
    # Codificar las etiquetas en one-hot encoding 
    encoder = OneHotEncoder(sparse=False)
    y_encoded = encoder.fit_transform(y.reshape(-1, 1))
    return y_encoded

Feature Extractor

Red sin Feature Extractor

In [ ]:

arquitectura = [4, 8, 5, 3]  # Capas ocultas de diferentes tamanos
red_neuronal = PerceptronMulticapa(capas=arquitectura, alpha=0.1)

# Codificar las etiquetas en formato one-hot encoding debido a que es multicapa
y_entrenamiento_encoded = encode_labels(y_entrenamiento)
y_prueba_encoded = encode_labels(y_prueba)

red_neuronal.entrenar(X_entrenamiento, y_entrenamiento_encoded, epochs=3000)
predicciones_encoded = red_neuronal.predecir(X_prueba)

# Decodificar las predicciones (seleccionar la clase con probabilidad mas alta)
predicciones = np.argmax(predicciones_encoded, axis=1)

accuracy = accuracy_score(y_prueba, predicciones)
precision = precision_score(y_prueba, predicciones, average='weighted')
recall = recall_score(y_prueba, predicciones, average='weighted')
f1 = f1_score(y_prueba, predicciones, average='weighted')
print("Metricas red especifica [4, 8, 5, 3]")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision (weighted): {precision:.2f}")
print(f"Recall (weighted): {recall:.2f}")
print(f"F1-score (weighted): {f1:.2f}")

Red con Feature Extractor

Comparacion entre Red sin Feature Extractor vs Red con Feature Extractor

Redes Convolucionales

Filtrado de imagenes offline

Filtro Bilateral

Red Convolucional 1

Red Convolucional 1 en imagenes crudas

Red Convolucional 1 con "Bilateral Filter"

Red Convolucional 2

Red Convolucional 2 en imagenes crudas

Red Convolucional 2 con "Bilateral Filter"

Tabla de Metricas

Respuesta a las preguntas
¿Cual es el tamano de los modelos?
¿Cual es la arquitectura (cuales y cuantas capas se utilizaron)?
¿Que ventajas tiene utilizar un feature extractor? referirse a las metricas y tiempos
¿Que modelo se entrena mas rapido?
¿Cuales secciones de la imagen son las mas importantes para la eleccion?
¿Cual fue el mejor modelo?